# VacationPy
----


In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

In [2]:
#Import csv data file from Weatherpy report and place in dataframe
csv_file = "Output/weather_data.csv"
heatmap_df=pd.read_csv(csv_file, encoding="utf-8")
heatmap_df.head()


,City,Cloudiness (%),Humidity (%),Temperature (F),Wind Speed (mph),Latitude,Longitude,Country,Date
0,Zabul Province,0,76,34.56,2.44,32.25,67.25,AF,1604110985
1,Horta,100,92,68.83,16.55,38.53,-28.63,PT,1604110985
2,Totness,42,84,81.30,13.24,5.88,-56.32,SR,1604110986
3,Jamestown,90,86,36.00,9.17,42.10,-79.24,US,1604110986
4,Hithadhoo,100,73,83.64,9.42,-0.60,73.08,MV,1604110951


### Humidity Heatmap


In [3]:
#
gmaps.configure(api_key=g_key)
locs = heatmap_df[["Latitude","Longitude"]]
humid = heatmap_df["Humidity (%)"]

fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locs, weights=humid, 
                                 dissipating=False, max_intensity=100, 
                                 point_radius=2.0)

fig.add_layer(heat_layer)

fig



Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
conditions_df = pd.DataFrame(heatmap_df, columns = ["City", "Temperature (F)", "Country", "Wind Speed (mph)", "Cloudiness (%)", "Latitude", "Longitude"])

ideal_temp = (conditions_df["Temperature (F)"] >=70) & (conditions_df["Temperature (F)"] <=80)
light_wind = conditions_df["Wind Speed (mph)"] <10
low_cloud = conditions_df["Cloudiness (%)"] ==0

few_cities = conditions_df[ideal_temp & light_wind & low_cloud]
len(few_cities)


13

### Hotel Map


In [5]:
hotel_df = few_cities[["City","Country","Latitude", "Longitude"]].copy()

hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Latitude,Longitude,Hotel Name
10,Rikitea,PF,-23.12,-134.97,
27,Verāval,IN,20.90,70.37,
76,Ratnagiri,IN,16.98,73.30,
113,Bouna,CI,9.27,-3.00,
114,Airai,TL,-8.93,125.41,
174,Belmonte,BR,-15.86,-38.88,
311,Sinazongwe,ZM,-17.26,27.46,
332,Salalah,OM,17.02,54.09,
356,Kati,ML,12.74,-8.07,
389,Bara,NG,10.37,10.73,


In [6]:
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

for index, row in hotel_df.iterrows():
    lat = row["Latitude"]
    lng = row["Longitude"]

    params["location"] = f"{lat},{lng}"

    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    response = requests.get(base_url, params=params)
    response2 = response.json()
    #results = response['results']

    try:
        hotel_df.loc[index, "Hotel Name"] = response2["results"][0]['name']
        #print(f"The nearest hotel in {city_name} is {results[0]['name']}.")

    except (KeyError, IndexError):
        print("No Matching Data")

hotel_df  

No Matching Data
No Matching Data
No Matching Data


,City,Country,Latitude,Longitude,Hotel Name
10,Rikitea,PF,-23.12,-134.97,Pension Maro'i
27,Verāval,IN,20.90,70.37,Lords Inn Somnath
76,Ratnagiri,IN,16.98,73.30,Hotel Landmark
113,Bouna,CI,9.27,-3.00,Hotel Reserve
114,Airai,TL,-8.93,125.41,Juvinal Martin
174,Belmonte,BR,-15.86,-38.88,Pousada Monte Carmelo O Pão
311,Sinazongwe,ZM,-17.26,27.46,Lakeview Lodge
332,Salalah,OM,17.02,54.09,HAMDAN PLAZA HOTEL
356,Kati,ML,12.74,-8.07,samassa la
389,Bara,NG,10.37,10.73,


In [7]:
hotel_df.head(1)

,City,Country,Latitude,Longitude,Hotel Name
10,Rikitea,PF,-23.12,-134.97,Pension Maro'i


In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [9]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)
fig

#Display figure


Figure(layout=FigureLayout(height='420px'))